# PGENFiles.jl

Routines for reading compressed storage of genotyped or imputed markers

[*Genome-wide association studies (GWAS)*](https://en.wikipedia.org/wiki/Genome-wide_association_study) data with imputed markers are often saved in the [**PGEN format**](https://www.cog-genomics.org/plink/2.0/input#pgen) in `.pgen` file.
It can store both hard calls and imputed data, unphased genotypes and phased haplotypes. Each variant is compressed separately. This is the central data format for [PLINK 2](https://www.cog-genomics.org/plink/2.0/). 

## Format description



## Installation

This package requires Julia v1.6 or later, which can be obtained from
https://julialang.org/downloads/ or by building Julia from the sources in the
https://github.com/JuliaLang/julia repository.


This package is registered in the default Julia package registry, and can be installed through standard package installation procedure: e.g., running the following code in Julia REPL.
```julia
using Pkg
pkg"add https://github.com/OpenMendel/PGENFiles.jl"
```


In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


## Type `Pgen`

The type `Pgen` is the fundamental type for .pgen-formatted files. It can be created using the following line.



In [2]:
using PGENFiles
p = Pgen(PGENFiles.datadir("bgen_example.16bits.pgen")) ;

This example file is a PGEN file converted from a BGEN file.

Number of variants and samples in the file is accessible with the functions `n_variants()` and `n_samples()`. 

In [3]:
println(n_variants(p))

199


In [4]:
println(n_samples(p))

500


## `VariantIterator`

Genotype information of each variant is compressed separately in PGEN files. The offsets (starting points in pgen file) of each variant can be inferred from the header. A way to iterate over variants in a PGEN file is provided through a `VariantIterator` object, created by the function `iterator()`. 

In [5]:
v_iter = iterator(p);

One may check index, offset (starting point of each variant record), record type (how each variant record is comprssed and what type of information it has), and length of each variant using the following code:

In [6]:
for v in v_iter
    println("Variant $(v.index): offset $(v.offset), type 0x$(string(v.record_type, base=16))," * 
        " length $(v.length)")
end

Variant 1: offset 617, type 0x60, length 1186
Variant 2: offset 1803, type 0x60, length 1182
Variant 3: offset 2985, type 0x41, length 1098
Variant 4: offset 4083, type 0x60, length 1182
Variant 5: offset 5265, type 0x60, length 1184
Variant 6: offset 6449, type 0x60, length 1186
Variant 7: offset 7635, type 0x44, length 1057
Variant 8: offset 8692, type 0x61, length 1146
Variant 9: offset 9838, type 0x61, length 1120
Variant 10: offset 10958, type 0x60, length 1184
Variant 11: offset 12142, type 0x61, length 1144
Variant 12: offset 13286, type 0x40, length 1125
Variant 13: offset 14411, type 0x41, length 1078
Variant 14: offset 15489, type 0x40, length 1125
Variant 15: offset 16614, type 0x41, length 1079
Variant 16: offset 17693, type 0x66, length 1115
Variant 17: offset 18808, type 0x60, length 1186
Variant 18: offset 19994, type 0x44, length 1056
Variant 19: offset 21050, type 0x60, length 1186
Variant 20: offset 22236, type 0x46, length 1060
Variant 21: offset 23296, type 0x61, le

More information on each variant is available in the attached `.pvar` file. 

## Genotypes and dosages

Genotypes of each variant is available through the function `get_genotypes()` or `get_genotypes!()`. For example, to obtain the genotypes of the first variant, one may do:

In [7]:
v = first(v_iter)
g, data, offset = get_genotypes(p, v)

(UInt8[0x03, 0x00, 0x00, 0x01, 0x00, 0x03, 0x01, 0x00, 0x03, 0x03  …  0x00, 0x01, 0x00, 0x01, 0x00, 0x00, 0x01, 0x00, 0x03, 0x01], UInt8[0x43, 0x1c, 0xff, 0x14, 0xc7, 0x0f, 0x00, 0x30, 0x01, 0x04  …  0xdc, 0x03, 0xd3, 0x42, 0x9e, 0x03, 0x07, 0x79, 0x4b, 0x3f], 0x000000000000007d)

`g` stores the genotypes, `data` is the variant record for `v`, and `offset` indicates where the track for genotypes ended. Encoding for `g` is as following:

| genotype code | genotype category | 
|:---:|:---:|
| `0x00` | homozygous REF | 
| `0x01` | heterozygous REF-ALT |
| `0x02` | homozygous ALT |
| `0x03` | missing |

To avoid array allocations for iterative genotype extraction, one may preallocate `g` and reuse it. As some of the variants are LD-compressed, an additional genotype buffer to keep the genotypes for the most recent non-LD-compressed variant may be desirable (`g_ld`). If `g_ld` is not provided, it will parse the genotypes of the most recent non-LD-compressed variant (stored in an internal dictionary) first.

For example:

In [8]:
g = Vector{UInt8}(undef, n_samples(p))
g_ld = similar(g)
for v in v_iter
    get_genotypes!(g, p, v; ldbuf=g_ld)
    v_rt = v.record_type & 0x07
    if v_rt != 0x02 && v_rt != 0x03 # non-LD-compressed. See Format description.
        g_ld .= g
    end
    
    # do someting with genotypes in `g`...
end

Similarly, ALT allele dosages are available through the function `alt_allele_dosage()` and `alt_allele_dosage!()`. As genotype information is required to retrieve dosages, space for genotypes are also required for `alt_allele_dosage!()`. These functions return dosages, parsed genotypes, and endpoint of the dosage information in the current variant record.

To obtain the dosages of the first variant: 

In [9]:
v = first(v_iter)
d, g, offset = alt_allele_dosage(p, v)

(Float32[NaN, 0.06427002, 0.08441162, 0.98254395, 0.08843994, 0.14111328, 1.0733032, 0.054138184, 0.10858154, 0.12310791  …  0.029785156, 0.9661255, 0.00079345703, 1.0126343, 0.042663574, 0.060302734, 1.0441284, 0.056518555, 1.8910522, 0.98895264], UInt8[0x03, 0x00, 0x00, 0x01, 0x00, 0x03, 0x01, 0x00, 0x03, 0x03  …  0x00, 0x01, 0x00, 0x01, 0x00, 0x00, 0x01, 0x00, 0x03, 0x01], 0x00000000000004a2)

Missing value is represented by a `NaN`. Code for a typical GWAS application should look like:

In [10]:
d = Vector{Float32}(undef, n_samples(p))
g = Vector{UInt8}(undef, n_samples(p))
g_ld = similar(g)
for v in v_iter
    alt_allele_dosage!(d, g, p, v; genoldbuf=g_ld)
    v_rt = v.record_type & 0x07
    if v_rt != 0x02 && v_rt != 0x03 # non-LD-compressed. See Format description.
        g_ld .= g
    end
    
    # do someting with dosage values in `d`...
end

## Speed

The current PGEN package can read in ~2000 variants / second for UK Biobank data, which is about 4x faster than reading in BGEN-formatted data. 